In [15]:
import tkinter as tk
from tkinter import *
from tkinter import ttk
from tkinter.scrolledtext import *
import tkinter.filedialog
from heapq import nlargest
from bs4 import BeautifulSoup
from urllib.request import urlopen
import heapq  
import spacy
nlp=spacy.load('en')
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS

In [16]:

def text_summarizer(raw_docx):
    raw_text = raw_docx
    docx = nlp(raw_text)
    stopwords = list(STOP_WORDS)
    word_frequencies = {}  
    for word in docx:  
        if word.text not in stopwords:
            if word.text not in word_frequencies.keys():
                word_frequencies[word.text] = 1
            else:
                word_frequencies[word.text] += 1


    maximum_frequncy = max(word_frequencies.values())

    for word in word_frequencies.keys():  
        word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)
        
    sentence_list = [ sentence for sentence in docx.sents ]

    
    sentence_scores = {}  
    for sent in sentence_list:  
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if len(sent.text.split(' ')) < 30:
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_frequencies[word.text.lower()]
                    else:
                        sentence_scores[sent] += word_frequencies[word.text.lower()]


    summarized_sentences = nlargest(7, sentence_scores, key=sentence_scores.get)
    final_sentences = [ w.text for w in summarized_sentences ]
    summary = ' '.join(final_sentences)
    return summary
  

In [17]:
window = Tk()
window.title("News Paper Summaryzer")
window.geometry("700x700")

def get_summary():
    raw_text = str(entry.get('1.0',tk.END))
    final_text = text_summarizer(raw_text)
    print(final_text)
    result = '\nSummary:{}'.format(final_text)
    tab1_display.insert(tk.END,result)



def get_text():
    raw_text = str(url_entry.get())
    page = urlopen(raw_text)
    soup = BeautifulSoup(page)
    fetched_text = ' '.join(map(lambda p:p.text,soup.find_all('p')))
    url_display.insert(tk.END,fetched_text)

def get_url_summary():
    raw_text = url_display.get('1.0',tk.END)
    final_text = text_summarizer(raw_text)
    result = '\nSummary:{}'.format(final_text)
    display_text.insert(tk.END,result)	


# URL 
l1=Label(window,text="Enter Newspaper URL To Summarize")
l1.grid(row=1,column=0)

raw_entry=StringVar()
url_entry=Entry(window,textvariable=raw_entry,width=50)
url_entry.grid(row=1,column=1)



button1=Button(window,text="Get URL",command=get_text, width=12)
button1.grid(row=4,column=1,padx=10,pady=10)

button2=Button(window,text="Summarize",command=get_url_summary, width=12)
button2.grid(row=5,column=1,padx=10,pady=10)
url_display = ScrolledText(window,height=10)
url_display.grid(row=7,column=0, columnspan=3,padx=5,pady=5)
display_text = ScrolledText(window,height=10)
display_text.grid(row=10,column=0, columnspan=3,padx=5,pady=5)

window.mainloop()

